## Datenquelle
http://www.offenedaten.frankfurt.de/dataset/frankfurter-stadtteilgrenzen-fur-gis-systeme

- imported into QGIS (https://www.qgis.org)
- exported as csv-Datei

In [2]:
%matplotlib inline

In [4]:
%%capture --no-display
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
!pip uninstall tensorflow-tensorboard -y
!pip install tensorflow geopy 
from geopy.geocoders import Nominatim
!pip install folium
%pip install --upgrade arcgis
import folium
from folium import plugins
from arcgis.gis import GIS
from arcgis.geocoding import geocode, reverse_geocode
from arcgis.geometry import Point

In [5]:
my_gis = GIS(verify_cert=False)


In [6]:
data = pd.read_csv('~/OneDrive/Stadtteile_Frankfurt.csv',sep=';',encoding='latin10')


df_sttl = data['STTLNAME']
df_ffm = []

for s in df_sttl:
    address = 'Frankfurt, '+s
    print(address)
    #geolocator = Nominatim(user_agent="myexplorer")
    location = geocode(address)
    latitude = location[0]['location']['y']
    longitude = location[0]['location']['x']
    print('The geograpical coordinate of {} are {}, {}.'.format(s,latitude, longitude))
    df_ffm.append([s,latitude,longitude])

df_ffm = pd.DataFrame(df_ffm,columns=['Stadtteil','Latitude','Longitude'])
df_ffm

Frankfurt, Altstadt
The geograpical coordinate of Altstadt are 50.111380000000054, 8.683560000000057.
Frankfurt, Innenstadt
The geograpical coordinate of Innenstadt are 50.11363000000006, 8.670690000000036.
Frankfurt, Bahnhofsviertel
The geograpical coordinate of Bahnhofsviertel are 50.10778000000005, 8.669590000000028.
Frankfurt, Westend-Süd
The geograpical coordinate of Westend-Süd are 50.12418000000008, 8.669160000000034.
Frankfurt, Westend-Nord
The geograpical coordinate of Westend-Nord are 50.12418000000008, 8.669160000000034.
Frankfurt, Nordend-West
The geograpical coordinate of Nordend-West are 50.12874000000005, 8.684350000000052.
Frankfurt, Nordend-Ost
The geograpical coordinate of Nordend-Ost are 50.12874000000005, 8.684350000000052.
Frankfurt, Ostend
The geograpical coordinate of Ostend are 50.11623000000003, 8.698150000000055.
Frankfurt, Bornheim
The geograpical coordinate of Bornheim are 50.13050000000004, 8.711160000000064.
Frankfurt, Gutleutviertel
The geograpical coordi

,Stadtteil,Latitude,Longitude
0,Altstadt,50.11138,8.68356
1,Innenstadt,50.11363,8.67069
2,Bahnhofsviertel,50.10778,8.66959
3,Westend-Süd,50.12418,8.66916
4,Westend-Nord,50.12418,8.66916
5,Nordend-West,50.12874,8.68435
6,Nordend-Ost,50.12874,8.68435
7,Ostend,50.11623,8.69815
8,Bornheim,50.13050,8.71116
9,Gutleutviertel,50.10307,8.66243


In [7]:
# create map of Frankfurt using latitude and longitude values
map_ffm = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough in zip(df_ffm['Latitude'], df_ffm['Longitude'], df_ffm['Stadtteil']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [float(lat), float(lng)],
        radius=5,
        popup=label,
        tooltip=borough,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_ffm)  
    
map_ffm

df_ffm[df_ffm['Stadtteil']=='Innenstadt']['Stadtteil']

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    lat_list=[]
    lng_list=[]
    popup_list=[]
    
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
        lat_list = [(v['venue']['location']['lat']) for v in results]
        lng_list = [(v['venue']['location']['lng']) for v in results]
        marker_list= list(zip(lat_list,lng_list))
        
        popup_list = [(v['venue']['name']) for v in results]
        
        folium.plugins.MarkerCluster(
            locations=marker_list,
            popups=popup_list
            ).add_to(map_ffm)

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood',
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [10]:
LIMIT = 100
borough_venues = getNearbyVenues(names=df_ffm['Stadtteil'],
                                   latitudes=df_ffm['Latitude'],
                                   longitudes=df_ffm['Longitude']
                                  )
map_ffm

Altstadt


NameError: name 'CLIENT_ID' is not defined

In [19]:
print(borough_venues.shape)
borough_venues.head()

(1185, 7)


,Neighbourhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Altstadt,50.11138,8.68356,Kleinmarkthalle,50.112778,8.682958,Market
1,Altstadt,50.11138,8.68356,Weinterasse Rollanderhof,50.112473,8.682164,Wine Bar
2,Altstadt,50.11138,8.68356,Römerberg,50.110489,8.682131,Plaza
3,Altstadt,50.11138,8.68356,SCHIRN Kunsthalle,50.110291,8.683542,Art Museum
4,Altstadt,50.11138,8.68356,Dom Aussichtsplattform,50.110609,8.684908,Scenic Lookout


In [20]:
borough_venues.groupby('Neighbourhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighbourhood,,,,,,
Altstadt,100,100,100,100,100,100
Bahnhofsviertel,100,100,100,100,100,100
Bergen-Enkheim,4,4,4,4,4,4
Berkersheim,3,3,3,3,3,3
Bockenheim,79,79,79,79,79,79
Bonames,8,8,8,8,8,8
Bornheim,38,38,38,38,38,38
Dornbusch,4,4,4,4,4,4
Eckenheim,10,10,10,10,10,10


In [21]:
print('There are {} uniques categories.'.format(len(borough_venues['Venue Category'].unique())))

There are 217 uniques categories.


In [22]:
uvc = borough_venues['Venue Category'].unique()
print( sorted(uvc))

['Accessories Store', 'African Restaurant', 'Airport', 'Airport Lounge', 'Airport Service', 'Airport Terminal', 'American Restaurant', 'Apple Wine Pub', 'Argentinian Restaurant', 'Art Museum', 'Arts & Crafts Store', 'Arts & Entertainment', 'Asian Restaurant', 'Austrian Restaurant', 'Auto Workshop', 'Automotive Shop', 'Bakery', 'Bank', 'Bar', 'Beach', 'Bed & Breakfast', 'Beer Bar', 'Beer Garden', 'Beer Store', 'Bistro', 'Board Shop', 'Boarding House', 'Boat or Ferry', 'Bookstore', 'Boutique', 'Breakfast Spot', 'Building', 'Burger Joint', 'Bus Stop', 'Business Service', 'Butcher', 'Cafeteria', 'Café', 'Cajun / Creole Restaurant', 'Castle', 'Chinese Restaurant', 'Chocolate Shop', 'Church', 'Cigkofte Place', 'Clothing Store', 'Cocktail Bar', 'Coffee Shop', 'College Residence Hall', 'Comedy Club', 'Concert Hall', 'Convenience Store', 'Cosmetics Shop', 'Coworking Space', 'Currywurst Joint', 'Czech Restaurant', 'Dance Studio', 'Deli / Bodega', 'Department Store', 'Dessert Shop', 'Diner', 'Dis

In [23]:
# one hot encoding
borough_onehot = pd.get_dummies(borough_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
borough_onehot['Neighbourhood'] = borough_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [borough_onehot.columns[-1]] + list(borough_onehot.columns[:-1])
borough_onehot = borough_onehot[fixed_columns]

borough_onehot.head()

,Neighbourhood,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Apple Wine Pub,Argentinian Restaurant,...,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Zoo
0,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
2,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Altstadt,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [24]:
borough_onehot.shape

(1185, 218)

In [25]:
pd.set_option('display.max_columns', None)
borough_grouped = borough_onehot.groupby('Neighbourhood').mean().reset_index()
borough_grouped

,Neighbourhood,Accessories Store,African Restaurant,Airport,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Apple Wine Pub,Argentinian Restaurant,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Austrian Restaurant,Auto Workshop,Automotive Shop,Bakery,Bank,Bar,Beach,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Bistro,Board Shop,Boarding House,Boat or Ferry,Bookstore,Boutique,Breakfast Spot,Building,Burger Joint,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Castle,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Clothing Store,Cocktail Bar,Coffee Shop,College Residence Hall,Comedy Club,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Currywurst Joint,Czech Restaurant,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Drugstore,Dry Cleaner,Duty-free Shop,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Film Studio,Food & Drink Shop,Food Court,Food Stand,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Friterie,Fruit & Vegetable Store,Furniture / Home Store,Garden,Gas Station,Gastropub,German Restaurant,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Hawaiian Restaurant,Health Food Store,Historic Site,History Museum,Home Service,Hookah Bar,Hostel,Hotel,Hotel Bar,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Insurance Office,Intersection,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Korean Restaurant,Lake,Laundry Service,Light Rail Station,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Malga,Market,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Modern European Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Museum,Music Store,Nightclub,Opera House,Organic Grocery,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Outlet Store,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Store,Pharmacy,Pie Shop,Pizza Place,Playground,Plaza,Pool,Pool Hall,Portuguese Restaurant,Pub,Radio Station,Ramen Restaurant,Record Shop,Rental Car Location,Restaurant,River,Rock Club,Sake Bar,Salad Place,Sandwich Place,Scenic Lookout,Science Museum,Seafood Restaurant,Shipping Store,Shopping Mall,Soccer Field,Soup Place,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stables,Steakhouse,Supermarket,Sushi Restaurant,Taco Place,Tapas Restaurant,Taverna,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Restaurant,Tibetan Restaurant,Toy / Game Store,Trail,Train Station,Tram Station,Trattoria/Osteria,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Waterfront,Whisky Bar,Wine Bar,Wine Shop,Zoo
0,Altstadt,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.030000,0.000000,0.000000,0.000000,0.01,0.000,0.000000,0.010000,0.000000,0.020000,0.000000,0.000000,0.01,0.000000,0.01,0.000000,0.01,0.000000,0.000000,0.000000,0.020000,0.010000,0.000000,0.030000,0.000000,0.000,0.000000,0.000000,0.110000,0.0,0.000000,0.010000,0.01,0.01,0.00,0.01,0.010000,0.020000,0.000000,0.000000,0.000000,0.00,0.010000,0.000000,0.010000,0.000000,0.000000,0.000000,0.02,0.000000,0.000000,0.000000,0.000000,0.010000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.010000,0.000000,0.000000,0.000000,0.000000,0.010000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.040000,0.000000,0.010000,0.000000,0.000000,0.000000,0.030000,0.000000,0.000000,0.00,0.000000,0.01,0.01,0.0,0.000000,0.000000,0.020000,0.000000,0.000000,0.010000,0.000000,0.010000,0.00,0.000000,0.000

In [26]:
borough_grouped.shape

(46, 218)

In [27]:
#url = 'https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data'
ffm_geo = "https://raw.githubusercontent.com/codeforamerica/click_that_hood/master/public/data/frankfurt-main.geojson"


folium.Choropleth(
    geo_data=ffm_geo,
    name='hotel',
    data=borough_grouped,
    columns=['Neighbourhood', 'Hotel'],
    key_on="feature.properties.name",
    fill_color='YlGn',
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name="Frequency of Hotels"
).add_to(map_ffm)

folium.Choropleth(
    geo_data=ffm_geo,
    name='trams',
    data=borough_grouped,
    columns=['Neighbourhood', 'Metro Station'],
    key_on="feature.properties.name",
    fill_color='RdPu',
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name="Frequency of Metro Stations"
).add_to(map_ffm)

folium.Choropleth(
    geo_data=ffm_geo,
    name='parks',
    data=borough_grouped,
    columns=['Neighbourhood', 'Park'],
    key_on="feature.properties.name",
    fill_color='YlOrBr',
    fill_opacity=0.4,
    line_opacity=0.2,
    legend_name="Frequency of Parks"
).add_to(map_ffm)

folium.LayerControl().add_to(map_ffm)

map_ffm

In [28]:
num_top_venues = 10

for hood in borough_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = borough_grouped[borough_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Altstadt----
                  venue  freq
0                  Café  0.11
1            Restaurant  0.05
2                 Plaza  0.05
3     German Restaurant  0.04
4          Burger Joint  0.03
5            Art Museum  0.03
6  Gym / Fitness Center  0.03
7      Department Store  0.02
8                   Pub  0.02
9              Boutique  0.02


----Bahnhofsviertel----
                        venue  freq
0                       Hotel  0.10
1           Indian Restaurant  0.07
2                         Bar  0.05
3                        Café  0.04
4          Chinese Restaurant  0.03
5                      Bakery  0.03
6       Vietnamese Restaurant  0.03
7          Seafood Restaurant  0.03
8                  Restaurant  0.03
9  Modern European Restaurant  0.02


----Bergen-Enkheim----
                        venue  freq
0           German Restaurant  0.50
1                 Supermarket  0.25
2       Outdoors & Recreation  0.25
3           Accessories Store  0.00
4             Organic Groc